In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
timeline = pd.read_csv('./crypto_reg_timeline.csv')
timeline['Date'] = pd.to_datetime(timeline['Date'], format='%Y%m%d')

In [3]:
timeline.head()

,Date,Country/Region,Topic
0,2018-06-11,UK,AML/CFT
1,2019-10-01,US,Cyrpto firms can become SPDIs
2,2019-12-16,HK,Risk assessment for VASPs and AML/CFT
3,2020-01-30,US,OCC issued cease-and-desist to Safra Bank for ...
4,2020-06-04,US,OCC fair access to banking services


In [4]:
tweets = pd.read_csv('Bitcoin_tweets_dataset_2.csv', lineterminator='\n')

In [5]:
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23 03:50:13,4680,2643,6232,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30 17:41:41,770,1145,9166,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False
2,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,576,1,0,False,2023-03-01 23:59:42,#Ethereum price update: \n\n#ETH $1664.02 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe\n\n#bitcoin #blockchai...,2010-03-26 10:15:26,236,1829,2195,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here\nAvailable on ios a...,['Bitcoin'],Twitter for Android,False
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16 09:57:56,12731,46,134,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...,"['Bitcoin', 'BTC']",Twitter Web App,False


In [6]:
engine = sqlite3.connect('./btc-data.db')
btc = pd.read_sql('SELECT * FROM BTCUSD', engine)

In [7]:
btc.head()

,date,open,high,low,close,volume,transactions,log,logRet
0,2023-11-22 06:15:00,36475.79,36492.0,36413.6,36443.80,82.165437,1840,10.503527,-0.000877
1,2023-11-22 06:00:00,36371.84,36490.0,36359.6,36475.79,188.767824,2105,10.504404,0.002854
2,2023-11-22 05:45:00,36406.20,36413.0,36341.2,36371.85,60.857387,2005,10.501550,-0.000944
3,2023-11-22 05:30:00,36410.40,36414.0,36355.2,36406.20,208.599722,2383,10.502494,0.000390
4,2023-11-22 05:15:00,36488.10,36488.2,36390.9,36392.00,96.059876,2276,10.502104,-0.001952
